In [1]:
import os
import math
import shutil
import warnings
import importlib
import urllib.request as request
from contextlib import closing
from numba import cuda

import cupy
import cudf
import cuml

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

import nvidia.cheminformatics.chembldata as chembldata

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [2]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=20000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    
def plot_kmeans_umap(df2):
    kmeans_float = cuml.KMeans(n_clusters=n_clusters)
    kmeans_float.fit(df2)

    umap = cuml.UMAP(n_neighbors=100,
                     a=1.0,
                     b=1.0,
                     learning_rate=1.0)
    Xt = umap.fit_transform(df2)

    df2['x'] = Xt[0]
    df2['y'] = Xt[1]
    df2['cluster'] = kmeans_float.labels_

    show_cluster_plot(df2)


# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
n_workers = len(client.scheduler_info()['workers'].keys())
client

Client Scheduler: ucx://127.0.0.1:53125 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 3 Cores: 3 Memory: 33.58 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [6]:
%%time
import nvidia.cheminformatics.chembldata
importlib.reload(nvidia.cheminformatics.chembldata)

# cache_directory = '/data/fp'
cache_directory = None
from nvidia.cheminformatics.fingerprint import MorganFingerprint

if cache_directory is None:
    chem_data = chembldata.ChEmblData(db_file=db_file, fp_type=MorganFingerprint)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()
df = dcudf.compute()

CPU times: user 2.26 s, sys: 542 ms, total: 2.8 s
Wall time: 15.6 s


In [7]:
# Compute Tanimoto similarities
fp = cupy.fromDlpack(df.to_dlpack())

# Tanimoto Similarity

In [17]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index#Other_definitions_of_Tanimoto_distance

@cuda.jit
def compute_norms(data, norms):
    i = cuda.grid(1)
    norms[i] = len(data[i])
    for j in range(len(data[i])):
        if data[i][j] != 0:
            data[i][j] = j + 1
            norms[i] = norms[i] + ((j + 1)**2)
    
    if norms[i] != 0:
        norms[i] = math.sqrt(norms[i])

@cuda.jit
def compute_tanimoto_sim_matix(data, norms, sim_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    a = data[i]
    b = data[j]
    
    prod = 0
    for k in range(len(data[i])):
        prod = prod + (a[k] * b[k])
        
    a_norm = norms[i]
    b_norm = norms[j]
    
    sim_array[i][j] = (prod / ((a_norm**2 + b_norm**2) - prod))
    
def tanimotoSimilarity(fp):
    norms = cupy.zeros(fp.shape[0])
    compute_norms.forall(norms.shape[0], 1)(fp, norms)

    sim_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_tanimoto_sim_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, norms, sim_array)
    return sim_array

# Jaccard Index Similarity

In [14]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index
@cuda.jit
def compute_jaccard_index_matix(data, index_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    a = data[i]
    b = data[j]
    
    intersects = 0
    unions = 0
    for k in range(len(data[i])):
        if a[k] == 1 and b[k] == 1:
            intersects += 1
            unions += 1
        elif a[k] == 1 or b[k] == 1:
            unions += 1

    index_array[i][j] = (intersects / unions)

def jaccardIndex(fp):
    index_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_jaccard_index_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, index_array)
    return index_array

# Clustering

In [18]:
%%time
sim_array = tanimotoSimilarity(fp)
# sim_array = jaccardIndex(fp)

df2 = cudf.DataFrame.from_gpu_matrix(sim_array)
plot_kmeans_umap(df2)

CPU times: user 1min 22s, sys: 1.56 s, total: 1min 23s
Wall time: 1min 21s
